In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm import tqdm
from keras.preprocessing import image

In [2]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_preprocess

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input as inception_resnet_v2_preprocess

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow_addons as tfa

In [4]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                 "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

In [5]:
binary_list = ["Black-grass", "Loose Silky-bent"]

In [6]:
width = 299
height = 299

num_aug = 1

In [7]:
apply_binary_classifier = {
    'xception': True,
    'inception_resnet_v2': True,
    'efficientnet': True,
    'cnn': True,
}

In [8]:
# Path to test images
test_dir = '../input/plant-seedlings-classification/'
test_dir_seg = '../input/plant-seedling-segmented/plant-seedling-segmented/'

# Path to saved models
model_dir = '../input/plant-seedling-models/'

In [9]:
def create_datagen(preprocess_fn, test_dir, test_path, width=299, height=299):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_fn)

    generator = datagen.flow_from_directory(
        directory=test_dir,
        classes=[test_path],
        target_size=(width, height),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical')
    
    return generator
    

In [10]:
def create_binary_datagen(preprocess_fn, test_dir, test_path, width=299, height=299):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_fn
    )

    generator = datagen.flow_from_directory(
        directory=test_dir,
        classes=[test_path],
        target_size=(width, height),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='binary')
    
    return generator

In [11]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow_addons as tfa

def create_binary_classifier():
    model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.efficientnet.EfficientNetB3(input_shape=(width, height, 3), include_top=False, weights='imagenet')(model_input)
    layer = model_main
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    
    layer = Dropout(0.2, name="top_dropout")(layer)

    model_dense2 = tf.keras.layers.Dense(256, activation = 'selu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(layer)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(1, activation="sigmoid")(model_dense2)

    model = tf.keras.models.Model(model_input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer=optimizer, metrics=["accuracy"])
    return model

In [12]:
def create_xception():
    model_main = Xception(weights='imagenet', input_shape=(width, height, 3), include_top=False)
    layer = model_main.output
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    
    layer = Dropout(0.2, name="top_dropout")(layer)

    dropout_1 = tf.keras.layers.Dropout(0.2)(layer)
    model_dense2 = tf.keras.layers.Dense(256, activation = 'selu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.2)(model_dense2)
    model_dense3 = tf.keras.layers.Dense(128, activation='selu')(dropout_2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(model_dense3)

    model = tf.keras.models.Model(model_main.input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [13]:
def create_inception_resnet_v2():
    model_input = tf.keras.layers.Input(shape=(width, height, 3), name='image_input')
    model_main = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=(width, height, 3), include_top=False, weights='imagenet')(model_input)
            
    model_dense1 = GlobalAveragePooling2D(name="avg_pool")(model_main)
    model_dense1 = BatchNormalization()(model_dense1)

    model_dense2 = tf.keras.layers.Dense(256, activation =  tfa.activations.mish, activity_regularizer=tf.keras.regularizers.l2(1e-5))(model_dense1)
    model_dense2 = BatchNormalization()(model_dense2)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [14]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten

def create_efficientnet():
    
    model_main = tf.keras.applications.efficientnet.EfficientNetB3(input_shape=(width, height, 3), include_top=False, weights='imagenet')
    
    layer = model_main.output
    layer = GlobalAveragePooling2D(name="avg_pool")(layer)
    layer = BatchNormalization()(layer)
    
    layer = Dropout(0.2, name="top_dropout")(layer)

    model_dense2 = tf.keras.layers.Dense(256, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(layer)
    dropout_2 = tf.keras.layers.Dropout(0.25)(model_dense2)
    model_out = tf.keras.layers.Dense(12, activation="softmax")(dropout_2)

    model = tf.keras.models.Model(model_main.input,  model_out)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [15]:
generator = create_binary_datagen(tf.keras.applications.efficientnet.preprocess_input, test_dir, 'test')
binary_classifier = create_binary_classifier()
binary_classifier.load_weights(f'{model_dir}binary_efficientnetB3_best.h5')
binary_pred = binary_classifier.predict(generator, steps=generator.samples)

class_list = []

for i in range(0, binary_pred.shape[0]):
    y_class = (binary_pred[i, :]>0.5).astype('int32')[0]
    class_list += [binary_list[y_class]]

binary = pd.DataFrame()
binary['file'] = generator.filenames
binary['file'] = binary['file'].str.replace(r'test/', '')
binary['species'] = class_list

binary.to_csv('binary_submission.csv', index=False)

Found 794 images belonging to 1 classes.


2022-04-18 17:46:23.997679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 17:46:24.086346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 17:46:24.087127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 17:46:24.089375: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

43950080/43941136 [==============================] - 0s 0us/step


2022-04-18 17:46:31.543941: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-18 17:46:34.172590: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [16]:
if apply_binary_classifier['xception']:
    
    datagen = create_datagen(xception_preprocess, test_dir_seg, 'seg_test')
    xception = create_xception()
    xception.load_weights(f'{model_dir}xception.h5')

    predictions = xception.predict(datagen, steps=datagen.samples)
    
    class_list = []

    for i in range(0, predictions.shape[0]):
        y_class = predictions[i, :].argmax(axis=-1)
        class_list += [species_list[y_class]]

    submission = pd.DataFrame()
    submission['file'] = datagen.filenames
    submission['file'] = submission['file'].str.replace(r'seg_test/', '')
    submission['species'] = class_list
    
    submission.to_csv('xception_submission.csv', index=False)
    
    for idx, pred in enumerate(submission['species']):
        if pred == 'Black-grass' or pred == 'Loose Silky-bent':
            print(f'Modifying Sumission {submission["species"][idx]} to {binary["species"][idx]}')
            submission['species'][idx] = binary['species'][idx]

    submission.to_csv('xception_binary_submission.csv', index=False)

Found 794 images belonging to 1 classes.
83697664/83683744 [==============================] - 0s 0us/step
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Su

In [17]:
if apply_binary_classifier['inception_resnet_v2']:

    datagen = create_datagen(inception_resnet_v2_preprocess, test_dir, 'test')
    
    inception_resnet_v2 = create_inception_resnet_v2()
    inception_resnet_v2.load_weights(f'{model_dir}inception_resnet_v2.h5')
    
    predictions = inception_resnet_v2.predict(datagen, steps=datagen.samples)
    
    class_list = []

    for i in range(0, predictions.shape[0]):
        y_class = predictions[i, :].argmax(axis=-1)
        class_list += [species_list[y_class]]

    submission = pd.DataFrame()
    submission['file'] = datagen.filenames
    submission['file'] = submission['file'].str.replace(r'test/', '')
    submission['species'] = class_list
    
    submission.to_csv('inception_resnet_v2_submission.csv', index=False)
    
    for idx, pred in enumerate(submission['species']):
        if pred == 'Black-grass' or pred == 'Loose Silky-bent':
            print(f'Modifying Sumission {submission["species"][idx]} to {binary["species"][idx]}')
            submission['species'][idx] = binary['species'][idx]

    submission.to_csv('inception_resnet_v2_binary_submission.csv', index=False)

Found 794 images belonging to 1 classes.
219070464/219055592 [==============================] - 1s 0us/step
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying 

In [18]:
if apply_binary_classifier['efficientnet']:

    datagen = create_datagen(tf.keras.applications.efficientnet.preprocess_input, test_dir, 'test')
    
    efficientnet = create_efficientnet()
    efficientnet.load_weights(f'{model_dir}efficientnetB3.h5')

    predictions = efficientnet.predict(datagen, steps=datagen.samples)

    class_list = []

    for i in range(0, predictions.shape[0]):
        y_class = predictions[i, :].argmax(axis=-1)
        class_list += [species_list[y_class]]

    submission = pd.DataFrame()
    submission['file'] = datagen.filenames
    submission['file'] = submission['file'].str.replace(r'test/', '')
    submission['species'] = class_list
    
    submission.to_csv('efficientnetb3_submission.csv', index=False)

    for idx, pred in enumerate(submission['species']):
        if pred == 'Black-grass' or pred == 'Loose Silky-bent':
            print(f'Modifying Sumission {submission["species"][idx]} to {binary["species"][idx]}')
            submission['species'][idx] = binary['species'][idx]
            
    submission.to_csv('efficientnetb3_binary_submission.csv', index=False)


Found 794 images belonging to 1 classes.
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission 

In [19]:
if apply_binary_classifier['cnn']:
    
    from keras.models import load_model
    datagen = ImageDataGenerator(
                             rotation_range=360,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True
                             )
    cnn = load_model(f'{model_dir}cnn.h5')
    
    submission = pd.DataFrame()

    for file in tqdm(os.listdir(f'{test_dir}test/')):
        image = cv2.imread(os.path.join(f'{test_dir}test/', file), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (128,128))
        image = np.expand_dims(image, axis=0)
        pred = np.zeros([12,])
        im = datagen.flow(image)
        pred = cnn.predict(im)[0]
        submission = submission.append({'file': file, 'species': species_list[np.where(pred==np.max(pred))[0][0]]}, ignore_index=True)
        
    submission.to_csv('cnn_submission.csv', index=False)
    
    for idx, pred in enumerate(submission['species']):
        if pred == 'Black-grass' or pred == 'Loose Silky-bent':
            print(f'Modifying Sumission {submission["species"][idx]} to {binary["species"][idx]}')
            submission['species'][idx] = binary['species'][idx]
            
    submission.to_csv('cnn_binary_submission.csv', index=False)


100%|██████████| 794/794 [00:48<00:00, 16.42it/s]


Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Black-grass to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Black-grass to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Black-grass to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Black-grass
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-bent to Loose Silky-bent
Modifying Sumission Loose Silky-be